In [1]:
!pip install pandas
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.2 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 42.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 23.8 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 64.4 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from PIL import Image
from transformers import AutoImageProcessor, ViTModel,ViTFeatureExtractor,AutoModel
from transformers import Dinov2Config, Dinov2Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


config=Dinov2Config(image_size=1800)
model_di=Dinov2Model(config)

#model_di = Dinov2Model.from_pretrained("facebook/dinov2-large")

model_di = AutoModel.from_pretrained("facebook/dinov2-base")
process=AutoImageProcessor.from_pretrained("facebook/dinov2-base")

#reeze parameter in dino
for param in model_di.parameters():
    param.requires_grad = False



#model_h = ViTModel.from_pretrained("facebook/dinov2-base")
#model_h = model_h.to(device)
# Move the model to GPU if available

# Define the custom dataset
class ImageOneDDataset(Dataset):
    def __init__(self, image_dir, label_file, transform=None):
        self.image_dir = image_dir
        self.label_file = label_file
        self.transform = transform

        # Get the list of JPEG files in the root directory
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpeg')]
        # Load the labels and one-D data
        self.labels = pd.read_excel(label_file).values
        self.labels =  self.labels



    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
         # Get the image file name
        image_file = self.image_files[idx]

        # Read the image from the file
        image = Image.open(os.path.join(self.image_dir, image_file)).convert('RGB')

        # Apply transformations to the image
        if self.transform:
            image = self.transform(image)

        # Get the label and one-D data
        label =torch.tensor(self.labels).T[idx+1]/100



        # Return the image, label, and one-D data
        return image.float(), label.float()

# Define the ResNet-MLP model
class V_transform(torch.nn.Module):
    def __init__(self):
        super(V_transform, self).__init__()

        self.model = model_di

        # Freeze the vit_b_32 parameters
        #for param in self.model.parameters():
         #   param.requires_grad = False

        self.fc = nn.Sequential(
          nn.Linear(in_features=768, out_features=500),
          nn.Sigmoid(),
          nn.Linear(in_features=500, out_features=50),
          nn.Sigmoid(),
          nn.Linear(in_features=50, out_features=20),
          nn.Sigmoid(),
          nn.Linear(in_features=20, out_features=6),
          nn.Sigmoid())



    def forward(self, images):
        #x=process(images)
        x = self.model(images).last_hidden_state[:, 0]  # Extract the CLS token
        x = self.fc(x)



        return x

# Create the dataset
dataset = ImageOneDDataset(image_dir='/workspace/low_fine',
                          label_file='/workspace/low_fine/select_low_fine.xlsx',
                          transform=transforms.Compose([
                              transforms.Resize((1350, 1800)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ]))
# Split the dataset into training, validation, and test sets
train_size = int(0.9 * len(dataset))
val_size = int(0.05 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create the data loaders
#Batch size depend on RAM of GPU
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# Create the model
model = V_transform()
model = model.to(device)
#multi GPU if have one GPU use 0
model= nn.DataParallel(model,device_ids=[0,1,2,3,4,5,6,7])


criterion = nn.MSELoss()

# Define the loss function and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model
for epoch in range(200):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        #model= nn.DataParallel(model,device_ids=[0,1])

        images = images.to(device)

        labels = labels.to(device)
        outputs = model(images)

        # Compute the loss
        loss = loss_fn(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Update the weights
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch: {epoch+1}, Batch: {i+1}, Loss: {loss.item()}')
# Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # Move the inputs and targets to the GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            val_loss += loss_fn(outputs, targets).item()

    # Print the validation loss
    val_loss /= len(val_loader)
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss}')


# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/workspace/model.pt')

Epoch: 1, Batch: 1, Loss: 0.15464678406715393
Epoch: 1, Validation Loss: 0.12640826404094696
Epoch: 2, Batch: 1, Loss: 0.12077894061803818
Epoch: 2, Validation Loss: 0.11071255058050156
Epoch: 3, Batch: 1, Loss: 0.11615234613418579
Epoch: 3, Validation Loss: 0.09700320661067963
Epoch: 4, Batch: 1, Loss: 0.11418741196393967
Epoch: 4, Validation Loss: 0.08498327434062958
Epoch: 5, Batch: 1, Loss: 0.10072320699691772
Epoch: 5, Validation Loss: 0.07420162856578827
Epoch: 6, Batch: 1, Loss: 0.09001656621694565
Epoch: 6, Validation Loss: 0.0643789991736412
Epoch: 7, Batch: 1, Loss: 0.07648468762636185
Epoch: 7, Validation Loss: 0.055930718779563904
Epoch: 8, Batch: 1, Loss: 0.06455881893634796
Epoch: 8, Validation Loss: 0.048702340573072433
Epoch: 9, Batch: 1, Loss: 0.0578109547495842
Epoch: 9, Validation Loss: 0.04262387007474899
Epoch: 10, Batch: 1, Loss: 0.04659997671842575
Epoch: 10, Validation Loss: 0.037579990923404694
Epoch: 11, Batch: 1, Loss: 0.042322736233472824
Epoch: 11, Validati

In [ ]:
torch.save(model.state_dict(), '/workspace/model.pt')

In [ ]:
def mape(y_true, y_pred):
    # Avoid division by zero
    y_true = torch.clamp(y_true, min=1e-8)
    # Compute the absolute percentage error
    ape = torch.abs((y_true - y_pred) / y_true)*100
    # Return the mean over all predictions
    return torch.mean(ape)

with torch.no_grad():
    test_loss = 0.0
    test_mape = 0.0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # Move the inputs and targets to the GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        test_loss += criterion(outputs, targets).item()
        # Compute the MAPE
        test_mape += mape(targets, outputs).item()

    # Print the test loss and MAPE
    test_loss /= len(test_loader)
    test_mape /= len(test_loader)
    print(f'Test Loss: {test_loss}')
    print(f'Test MAPE: {test_mape}')

In [ ]:
mape(targets, outputs)

tensor(25.3811, device='cuda:0')

In [ ]:
inputs[0].shape

torch.Size([3, 1000, 1000])

In [ ]:
from torchviz import make_dot
# Assume you have a model called net and some input called images
output=model(images, one_d_data)
make_dot(output, params=dict(list(model.named_parameters()))).render("model", format="png")

'model.png'

In [ ]:
!pip install hiddenlayer

In [ ]:
import hiddenlayer as hl
# Assume you have a model called net and some input called images
graph = hl.build_graph(model,one_d_data)
graph.save("model", format="png")

TypeError: ignored